In [1]:
__file__ = "__init__.py"

In [ ]:
import sys, os
import re
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from typing import Dict, List, Optional
from rapidfuzz import fuzz, process
from sentence_transformers import SentenceTransformer

project_root = Path(__file__).resolve().parents[1]
sys.path.append(str(project_root))

from utils.ncomp import rlst, srlst, clst, glst, rrlst, dtlst, sslst

paths = {
    "processed": os.path.abspath(f"{project_root}/data/storage/processed"),
    "odata": os.path.abspath(f"{project_root}/data/storage/processed/final_cleaning.csv"),
}
spec = pd.read_csv(f"{paths['processed']}/final_cleaning.csv")

ModuleNotFoundError: No module named 'utils'

In [ ]:
class NeuralNetwork(nn.Module):
    """
    Neural Network nâng cấp với Batch Normalization và Dropout để tinh chỉnh embeddings.
    
    Args:
        input_size (int): Kích thước đầu vào.
        hidden_size (int): Số nơ-ron ở lớp ẩn.
        num_classes (int): Kích thước đầu ra (trong trường hợp này bằng input_size).
        dropout_rate (float): Tỉ lệ dropout.
    """
    def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.3):
        super(NeuralNetwork, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout1(out)
        out = self.l2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout2(out)
        out = self.l3(out)
        return out

# ---------------------------
# ComponentExtractor với Ensemble và NN refinement
# ---------------------------
class ComponentExtractor:
    def __init__(
        self,
        threshold: float = 0.45,
        fuzzy_threshold: int = 60,
        use_nn: bool = True,
        embedding_dim: int = 384,  # kích thước của embedding từ SentenceTransformer "all-MiniLM-L6-v2"
        hidden_size: int = 256
    ) -> None:
        # Load spec từ file (ở đây giả sử spec đã được load từ paths["odata"])
        # Chúng ta không sử dụng "brand" trong trích xuất vì nó được lấy từ spec.
        self.spec = None  # Nếu cần, spec sẽ được sử dụng bởi lớp PostProcessor riêng.
        
        # Khai báo 2 mô hình SentenceTransformer (ensemble)
        self.models = [
            SentenceTransformer("all-mpnet-base-v2"),
            SentenceTransformer("all-MiniLM-L6-v2")
        ]
        # Các danh mục cần trích xuất (không bao gồm "brand")
        self.components = {
            "gpu": sorted(glst(), key=len, reverse=False),
            "cpu": sorted(clst(), key=len, reverse=False),
            "ram": sorted(rlst(), key=len, reverse=False),
            "resolution": sorted(srlst(), key=len, reverse=True),
            "refresh rate": sorted(rrlst(), key=len, reverse=False),
            "display type": sorted(dtlst(), key=len, reverse=False),
            "screen size": sorted(sslst(), key=len, reverse=False),
        }
        # Tạo DataFrame từ dictionary (các cột dạng chữ thường)
        self.df_components = pd.DataFrame.from_dict(self.components, orient="index").transpose()
        self.df_components.columns = [col.lower() for col in self.df_components.columns]
        
        # Pre-compute embeddings cho từng danh mục cho từng mô hình
        self.embeddings = []  # danh sách chứa dictionary embeddings cho mỗi mô hình
        for model in self.models:
            model_emb = {}
            for comp_name, comp_list in self.components.items():
                cleaned = [self._clean_text(text) for text in comp_list]
                model_emb[comp_name] = model.encode(cleaned, convert_to_tensor=True)
            self.embeddings.append(model_emb)
        
        self.threshold = threshold
        self.fuzzy_threshold = fuzzy_threshold
        self.use_nn = use_nn
        if self.use_nn:
            self.embedding_nets = {}
            for comp_name in self.components.keys():
                # Khởi tạo một NeuralNetwork cho mỗi danh mục
                self.embedding_nets[comp_name] = NeuralNetwork(input_size=embedding_dim, hidden_size=hidden_size, num_classes=embedding_dim, dropout_rate=0.3)
                # Ở chế độ inference, ta để chúng ở eval; khi fine-tuning có thể đặt train()
                self.embedding_nets[comp_name].eval()
        
    def _clean_text(self, text: str) -> str:
        """Chuyển thành chữ thường, thay '-' và '/' bằng khoảng trắng, loại bỏ ký tự không cần thiết."""
        text = text.lower()
        text = re.sub(r"[\-/]", " ", text)
        text = re.sub(r"[^a-z0-9\s]", "", text)
        text = re.sub(r"\s+", " ", text).strip()
        return text

    def _fuzzy_match(self, candidates: List[str], query: str) -> Optional[str]:
        """Sử dụng fuzzy matching để tìm candidate tốt nhất từ danh sách."""
        best_candidate = None
        best_score = 0
        for cand in candidates:
            score = fuzz.partial_ratio(query, cand)
            if score > best_score:
                best_score = score
                best_candidate = cand
        if best_score >= self.fuzzy_threshold:
            return best_candidate
        return None

    def _ensemble_cosine(self, comp_name: str, query_embedding_list: List[torch.Tensor]) -> torch.Tensor:
        """Tính trung bình trọng số (ensemble) của cosine similarity từ các mô hình."""
        # Lấy danh sách điểm cosine từ từng mô hình
        scores_list = []
        for idx, model_emb in enumerate(self.embeddings):
            candidate_emb = model_emb[comp_name]
            # Nếu sử dụng NN refinement, chuyển đổi candidate embedding
            if self.use_nn:
                net = self.embedding_nets[comp_name]
                candidate_emb = net(candidate_emb)
                query_emb = net(query_embedding_list[idx])
            else:
                query_emb = query_embedding_list[idx]
            # Tính cosine similarity
            dot = torch.matmul(candidate_emb, query_emb.unsqueeze(1)).squeeze()
            norm_candidate = torch.norm(candidate_emb, dim=1)
            norm_query = torch.norm(query_emb)
            cos_scores = dot / (norm_candidate * norm_query + 1e-8)
            scores_list.append(cos_scores)
        # Kết hợp theo trọng số, ví dụ: [0.6, 0.4]
        weights = [0.6, 0.4]
        combined = sum(w * s for w, s in zip(weights, scores_list))
        return combined

    def extract_components(self, new_question: str) -> Dict[str, Optional[str]]:
        """
        Trích xuất các component từ new_question.
        Sử dụng ensemble của các mô hình SentenceTransformer, kết hợp với NN refinement nếu enabled.
        Nếu cosine similarity không đạt, fallback qua fuzzy matching.
        """
        cleaned_query = self._clean_text(new_question)
        # Tính embedding của câu hỏi từ mỗi mô hình
        query_embeddings = [model.encode(cleaned_query, convert_to_tensor=True) for model in self.models]
        # Đưa tất cả query embeddings về cùng device với các embeddings (giả sử model đầu tiên)
        device = self.embeddings[0][next(iter(self.embeddings[0]))].device
        query_embeddings = [emb.to(device) for emb in query_embeddings]
        
        results = {}
        for comp_name, candidates in self.components.items():
            if not candidates:
                results[comp_name] = None
                continue
            combined_scores = self._ensemble_cosine(comp_name, query_embeddings)
            best_score, best_idx = torch.max(combined_scores, dim=0)
            candidate = candidates[best_idx.item()] if best_score.item() >= self.threshold else None
            if candidate is None:
                candidate = self._fuzzy_match(candidates, new_question.lower())
            results[comp_name] = candidate
        return results

In [ ]:
class PostProcessor:
    def __init__(self) -> None:
        # Đọc file spec và chuyển tên cột về chữ thường
        self.spec = pd.read_csv(paths["odata"])
        self.spec.columns = [col.lower() for col in self.spec.columns]

        for col in ["gpu", "cpu", "brand", "ram", "resolution", "refresh rate", "display type", "screen size"]:
            if col in self.spec.columns:
                self.spec[col] = self.spec[col].astype(str).str.lower()

    def process_gpu(self, detected_gpu: str, new_question: str) -> list:
        """
        Xử lý GPU:
        - Nếu detected_gpu (ví dụ: "geforce rtx 3070 ti") xuất hiện nguyên vẹn trong new_question, 
            thì sử dụng nó để lọc trong spec.
        - Nếu không, tách thành các token và sử dụng cửa sổ trượt để tìm chuỗi con dài nhất xuất hiện trong new_question.
        - Sau đó, lọc trong spec theo chuỗi con tìm được và trả về danh sách các tên GPU tương ứng.
        """
        new_q = new_question.lower()
        if not detected_gpu:
            return None
        
        # Nếu detected_gpu xuất hiện nguyên vẹn trong new_question
        if detected_gpu in new_q:
            filtered_df = self.spec[self.spec["gpu"].str.contains(detected_gpu, case=False, na=False)]
            if not filtered_df.empty:
                return filtered_df["gpu"].unique().tolist()
            else:
                return [detected_gpu]
        
        # Nếu không, tách detected_gpu thành các token
        tokens = detected_gpu.split()
        best_candidate = ""
        
        # Duyệt từng vị trí bắt đầu
        for i in range(len(tokens)):
            candidate = ""
            # Duyệt cửa sổ từ vị trí i đến hết
            for j in range(i, len(tokens)):
                candidate = " ".join(tokens[i:j+1])
                if candidate in new_q:
                    # Lưu lại candidate nếu dài hơn candidate đã lưu
                    if len(candidate) > len(best_candidate):
                        best_candidate = candidate
                else:
                    # Nếu candidate không có, break cửa sổ hiện tại
                    break

        if best_candidate:
            filtered_df = self.spec[self.spec["gpu"].str.contains(best_candidate, case=False, na=False)]
            if not filtered_df.empty:
                return filtered_df["gpu"].unique().tolist()
            else:
                return [best_candidate]
        return None

    def process_cpu(self, detected_cpu: str, new_question: str) -> list:
        """
        Loại bỏ "th" và "gen", sau đó kiểm tra nếu detected_cpu không có trong new_question,
        tách thành các token và duyệt dần để tạo chuỗi con.
        Sau đó, lọc trong spec theo cột 'cpu' và chỉ trả về danh sách cuối cùng.
        """
        new_q = new_question.lower()
        if not detected_cpu:
            return None
        normalized = detected_cpu.lower().replace("th", "").replace("gen", "").strip()
        filtered_df = self.spec[self.spec["cpu"].str.contains(normalized, case=False, na=False)]
        if normalized in new_q:
            filtered_df = self.spec[self.spec["cpu"].str.contains(normalized, case=False, na=False)]
            if not filtered_df.empty:
                return filtered_df["cpu"].unique().tolist()
        tokens = normalized.split()
        candidate = ""
        found_candidate = None
        for token in tokens:
            candidate = (candidate + " " + token).strip()
            if candidate in new_q:
                found_candidate = candidate
            else:
                break
        if found_candidate:
            filtered_df = self.spec[self.spec["cpu"].str.contains(found_candidate, case=False, na=False)]
            if not filtered_df.empty:
                return filtered_df["cpu"].unique().tolist()
        return filtered_df["cpu"].unique().tolist() if not filtered_df.empty else None

    def process_resolution(self, detected_res: str) -> str:
        """
        Xử lý resolution:
          - Loại bỏ các từ khóa như "display", "resolution", ...
          - Sử dụng kỹ thuật fuzzy matching để xác định key trong dict.
        """
        if not detected_res:
            return None
        keywords = ["display", "resolution", "display panel", "display resolution", "screen resolution", "monitor resolution"]
        comp = detected_res.lower()
        for kw in keywords:
            comp = comp.replace(kw, "")
        comp = comp.strip()
        resolution_dict = {
            "3072 x 1920": ["3072 x 1920", "3k", "3072p", "triple hd"],
            "1920 x 1200": ["1920 x 1200", "wuxga", "16 10 hd+", "hd+ 16 10"],
            "2560 x 1600": ["2560 x 1600", "wqxga", "quad extended 16 10", "retina-like"],
            "2560 x 1440": ["2560 x 1440", "qhd", "quad hd", "2k", "wqhd"],
            "1920 x 1080": ["1920 x 1080", "fhd", "full hd", "1080p"],
            "3840 x 2160": ["3840 x 2160", "4k uhd", "4k", "uhd", "ultra hd", "2160p"],
            "2880 x 1800": ["2880 x 1800", "retina 15", "qhd+ 16 10"],
            "3840 x 2400": ["3840 x 2400", "wquxga", "16 10 4k+"],
            "3200 x 2000": ["3200 x 2000", "qhd+", "3k2k", "wqxga 16 10"],
            "2880 x 1620": ["2880 x 1620", "qhd+ 16 9", "16 9 qhd+", "3k2k 16 9"],
            "3456 x 2160": ["3456 x 2160", "retina 16", "16 inch retina", "3.5k"],
            "2400 x 1600": ["2400 x 1600", "qxga+"],
        }
        best_match = None
        best_score = 0
        for key, synonyms in resolution_dict.items():
            for syn in synonyms:
                score = fuzz.partial_ratio(comp, syn)
                if score > best_score:
                    best_score = score
                    best_match = key
        return best_match if best_score >= 50 else None

    def process_refresh_rate(self, detected_rr: str) -> str:
        """
        Lấy số từ refresh rate.
        """
        if not detected_rr:
            return None
        match = re.search(r'(\d+)', detected_rr)
        return match.group(1) if match else None

    def process_screen_size(self, detected_ss: str) -> float:
        """
        Loại bỏ "inch" và chuyển thành float.
        """
        if not detected_ss:
            return None
        comp = detected_ss.lower().replace("inch", "").strip()
        try:
            return float(comp)
        except:
            return None

    def postprocess(self, detected_components: dict, new_question: str) -> dict:
        """
        Tổng hợp quá trình hậu xử lý:
          - Áp dụng các hàm process_gpu, process_cpu, process_resolution, process_refresh_rate, process_screen_size.
          - Nếu không tìm được phù hợp, trả về None cho component đó.
        """
        output = {}
        output["brand"] = detected_components.get("brand")
        output["gpu"] = self.process_gpu(detected_components.get("gpu"), new_question)
        output["cpu"] = self.process_cpu(detected_components.get("cpu"), new_question)
        output["ram"] = detected_components.get("ram")  
        output["resolution"] = self.process_resolution(detected_components.get("resolution"))
        output["refresh rate"] = self.process_refresh_rate(detected_components.get("refresh rate"))
        dt = detected_components.get("display type")
        output["display type"] = dt.lower().strip() if dt else None
        output["screen size"] = self.process_screen_size(detected_components.get("screen size"))
        
        return output

In [ ]:
extractor = ComponentExtractor(threshold=0.35, fuzzy_threshold=60)
new_question = "give me a laptop asus have rtx 4070, intel core i7 13th and ram 16gb , 512gb ssd"
detected = extractor.extract_components(new_question)
print("Detected Components:", detected)

Detected Components: {'brand': 'asus', 'gpu': 'geforce rtx 4070', 'cpu': 'intel core i7 12th', 'ram': 'ram 16gb', 'resolution': '2560 x 1600', 'refresh rate': None, 'display type': 'lcd', 'screen size': None}


In [ ]:
post_processor = PostProcessor()
final_components = post_processor.postprocess(detected, new_question)
print("Final Processed Components:", final_components)

Final Processed Components: {'brand': 'asus', 'gpu': ['nvidia geforce rtx 4070'], 'cpu': ['intel core i7 12700h', 'intel core i7 13700h', 'intel core i7 13700hx', 'intel core i7 12650h', 'intel core i7 11800h', 'intel core i7 13650hx', 'intel core i7 10870h', 'intel core i7 13620h', 'intel core i7 14650hx', 'intel core i7 14700hx', 'intel core i7 9750h', 'intel core i7 12800hx', 'intel core i7 1355u', 'intel core i7 1365u', 'intel core i7 1165g7', 'intel core i7 11850h', 'intel core i7 13800h', 'intel core i7 1260p', 'intel core i7 1360p', 'intel core i7 10610u', 'intel core i7 12800h', 'intel core i7 10750h', 'intel core i7 10875h', 'intel core i7 1185g7'], 'ram': 'ram 16gb', 'resolution': '2560 x 1600', 'refresh rate': None, 'display type': 'lcd', 'screen size': None}
